In [1]:
import pandas as pd
import os
import re

In [4]:
folders = [f for f in os.listdir('..') if 'graph-type' in f]
folders

['graph500_graph-type__1048576_vertices__16777216_edges__182_root__2_iter__32_tile-groups__1_tgx__1_tgy',
 'graph500_graph-type__1048576_vertices__16777216_edges__182_root__1_iter__64_tile-groups__1_tgx__1_tgy',
 'graph500_graph-type__1048576_vertices__16777216_edges__182_root__6_iter__128_tile-groups__1_tgx__1_tgy',
 'graph500_graph-type__1048576_vertices__16777216_edges__182_root__1_iter__128_tile-groups__1_tgx__1_tgy',
 'graph500_graph-type__1048576_vertices__16777216_edges__182_root__2_iter__64_tile-groups__1_tgx__1_tgy',
 'graph500_graph-type__1048576_vertices__16777216_edges__182_root__6_iter__64_tile-groups__1_tgx__1_tgy',
 'graph500_graph-type__1048576_vertices__16777216_edges__182_root__1_iter__1_tile-groups__1_tgx__1_tgy',
 'graph500_graph-type__1048576_vertices__16777216_edges__182_root__1_iter__32_tile-groups__1_tgx__1_tgy',
 'graph500_graph-type__1048576_vertices__16777216_edges__182_root__6_iter__32_tile-groups__1_tgx__1_tgy',
 'graph500_graph-type__1048576_vertices__1677

In [16]:
def vcache_name(x,y):
    return "replicant_tb_top.testbench.DUT.py[0].podrow.px[0].pod.north_vc_x[0].north_vc_row.vc_y[%d].vc_x[%d].vc.cache.vcache_prof" % (y,x)

GROUP_BY_COLUMNS = [
    'time',
    'tag',
    'global_ctr',
    'graph_type',
    'vertices',
    'edges',
    'root',
    'iteration',
    'tile_groups',
    'tgx',
    'tgy',
]

def vcache(vcaches_data):
    # get the min and max time
    min_time = vcaches_data['time'].min()
    max_time = vcaches_data['time'].max()
    # get the min and max global ctr
    min_global_ctr = vcaches_data['global_ctr'].min()
    max_global_ctr = vcaches_data['global_ctr'].max()
    # just keep the min and max time
    vcaches_data = vcaches_data[(vcaches_data['time']==min_time)|(vcaches_data['time']==max_time)]
    summary = vcaches_data.groupby(GROUP_BY_COLUMNS).sum().diff().dropna()
    # mark time
    summary['time_start'] = min_time
    summary['time_end'] = max_time
    summary['time_elapsed'] = max_time-min_time
    # mark cycles
    summary['global_ctr_start'] = min_global_ctr
    summary['global_ctr_end'] = max_global_ctr
    summary['global_ctr_elapsed'] = max_global_ctr-min_global_ctr
    return summary
    
def vcore(vcores_data):
    full_summary = pd.DataFrame()
    for x in range(16):
        for y in range(8):
            # filter only this vcore
            vcore_data = vcores_data[(vcores_data['x']==x)&(vcores_data['y']==y)]
            if vcore_data.empty: continue
            # get the min and max time
            min_time = vcore_data['time'].min()
            max_time = vcore_data['time'].max()
            # get the min and max global ctr
            min_global_ctr = vcore_data['global_ctr'].min()
            max_global_ctr = vcore_data['global_ctr'].max()
            # just keep the min and max time    
            vcore_data = vcore_data[(vcore_data['time']==min_time)|(vcore_data['time']==max_time)]
            summary = vcore_data.groupby(GROUP_BY_COLUMNS+['x','y']).sum().diff().dropna()
            # mark time
            summary['time_start'] = min_time
            summary['time_end'] = max_time
            summary['time_elapsed']=max_time-min_time
            # mark cycles
            summary['global_ctr_start'] = min_global_ctr
            summary['global_ctr_end'] = max_global_ctr
            summary['global_ctr_elapsed'] = max_global_ctr-min_global_ctr
            # append
            full_summary = full_summary.append(summary)
    # return the full summary
    return full_summary

def folder(fldr):
    # parse the folder name to get experiment parameters
    m = re.match(r'^(graph500|uniform)_graph-type__(\d+)_vertices__(\d+)_edges__(\d+)_root__(\d+)_iter__(\d+)_tile-groups__(\d+)_tgx__(\d+)_tgy$', fldr)
    graph_type = m.group(1)
    vertices = int(m.group(2))
    edges = int(m.group(3))
    root = int(m.group(4))
    iteration = int(m.group(5))
    tile_groups = int(m.group(6))
    tgx = int(m.group(7))
    tgy = int(m.group(8))
    # read data from experiment
    vcaches_data_file = '../' + fldr + '/vcache_stats.csv'
    vcores_data_file  = '../' + fldr + '/vanilla_stats.csv'
    vcaches_data = pd.read_csv(vcaches_data_file) 
    vcores_data = pd.read_csv(vcores_data_file)
    # post process vcache data    
    vcaches_data['graph_type']=graph_type
    vcaches_data['vertices']=vertices
    vcaches_data['edges']=edges
    vcaches_data['root']=root
    vcaches_data['iteration']=iteration
    vcaches_data['tile_groups']=tile_groups
    vcaches_data['tgx']=tgx
    vcaches_data['tgy']=tgy
    vcache_data = vcache(vcaches_data)
    # post process vcore data
    vcores_data['graph_type']=graph_type
    vcores_data['vertices']=vertices
    vcores_data['edges']=edges
    vcores_data['root']=root
    vcores_data['iteration']=iteration
    vcores_data['tile_groups']=tile_groups
    vcores_data['tgx']=tgx
    vcores_data['tgy']=tgy
    vcore_data = vcore(vcores_data)
    return (vcache_data, vcore_data)

folder(folders[0])

(                                                                                                  instr_ld  \
 time       tag        global_ctr graph_type vertices edges    root iteration tile_groups tgx tgy             
 2950782000 3410231472 2950766    graph500   1048576  16777216 182  2         32          1   1    203122.0   
 
                                                                                                   instr_ld_ld  \
 time       tag        global_ctr graph_type vertices edges    root iteration tile_groups tgx tgy                
 2950782000 3410231472 2950766    graph500   1048576  16777216 182  2         32          1   1            0.0   
 
                                                                                                   instr_ld_ldu  \
 time       tag        global_ctr graph_type vertices edges    root iteration tile_groups tgx tgy                 
 2950782000 3410231472 2950766    graph500   1048576  16777216 182  2         32          1

In [17]:
vcache_summary_data = pd.DataFrame()
vcore_summary_data = pd.DataFrame()
for fldr in folders:
    cache,core = folder(fldr)
    vcache_summary_data = vcache_summary_data.append(cache)
    vcore_summary_data = vcore_summary_data.append(core)
vcore_summary_data

,,,,,,,,,,,,,pc_r,pc_n,cycle,instr_total,instr_fadd,instr_fsub,instr_fmul,instr_fsgnj,instr_fsgnjn,instr_fsgnjx,...,stall_fcsr,stall_remote_ld_wb,stall_ifetch_wait,stall_remote_flw_wb,time_start,time_end,time_elapsed,global_ctr_start,global_ctr_end,global_ctr_elapsed
time,tag,global_ctr,graph_type,vertices,edges,root,iteration,tile_groups,tgx,tgy,x,y,,,,,,,,,,,,,,,,,,,,,
1075026000,3359637504,1075010,graph500,1048576,16777216,182,2,32,1,1,0,0,51.0,51.0,949596.0,26905.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,125430000,1075026000,949596000,125414,1075010,949596
1707936000,3376414736,1707920,graph500,1048576,16777216,182,2,32,1,1,0,1,52.0,52.0,1582212.0,47180.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,125724000,1707936000,1582212000,125708,1707920,1582212
1678516000,3393191968,1678500,graph500,1048576,16777216,182,2,32,1,1,0,2,53.0,53.0,1552461.0,45011.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,126055000,1678516000,1552461000,126039,1678500,1552461
1978381000,3409969200,1978365,graph500,1048576,16777216,182,2,32,1,1,0,3,54.0,92.0,1851944.0,61574.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,126437000,1978381000,1851944000,126421,1978365,1851944
1373266000,3426746432,1373250,graph500,1048576,16777216,182,2,32,1,1,0,4,93.0,93.0,1246419.0,36870.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,126847000,1373266000,1246419000,126831,1373250,1246419
1165650000,3443523664,1165634,graph500,1048576,16777216,182,2,32,1,1,0,5,94.0,94.0,1038364.0,28535.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,127286000,1165650000,1038364000,127270,1165634,1038364
1954973000,3460300896,1954957,graph500,1048576,16777216,182,2,32,1,1,0,6,95.0,95.0,1827194.0,57378.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,127779000,1954973000,1827194000,127763,1954957,1827194
1967690000,3477078128,1967674,graph500,1048576,16777216,182,2,32,1,1,0,7,96.0,97.0,1839398.0,56728.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,128292000,1967690000,1839398000,128276,1967674,1839398
1866456000,3359899776,1866440,graph500,1048576,16777216,182,2,32,1,1,1,0,52.0,52.0,1737851.0,58093.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,128605000,1866456000,1737851000,128589,1866440,1737851


In [18]:
vcache_summary_data.to_csv('vcache_summary.txt')

In [19]:
vcore_summary_data.to_csv('vcore_summary.txt')